# Linear and Non Linear models

In [ ]:
import torch
import seaborn as sns

## 1 Linear models
One of the simplest models is the linear model. The basic shape of the formula is:

$$
y = wx + b
$$
Here, $x$ is a variable, $w$ and $b$ are constants that determine the slope and intercept of the line.

We can scale this formula for multiple variables, like this:
$$
y = w_1 x_1 + w_2 x_2 + \dots + w_n x_n + b
$$

Because it is easier to do this calculation as a dotproduct, we will typically use matrices.

So, let us have $n$ observations of $m$ features, and we will represent that in a matrix $X$ with dimensions $(n,m)$. We will generate random observations:

In [ ]:
n = 100
m = 3
X = torch.rand(n, m)
X.shape

This matrix $X$ is our data. Now we want to have $m$ weights, and we want to sum that up untill we have a single number, so our weight $W$ will have dimensions $(m, 1)$. To get matrix multiplication to work, we need the last dimension of $X$ and the first dimension of $W$ to match. So because $X$ has 3 features (last dimension of $X$)$ our $W$ needs to have 3 numbers as its first dimension for `jnp.dot(X, W)` to work.

In [ ]:
W = torch.normal(0., 0.1, (m, 1))
b = torch.normal(0, 0.1, (1,))
W, b

In [ ]:
W.shape, b.shape

Instead of using a forloop to calculate this linear formula $n$ times, we will use a dotproduct. This will calcute 

$$ 
y = w_1 x_1 + w_2 x_2 + w_3 x_3 + b
$$

for all $n$ observations.

In [ ]:
yhat = torch.matmul(X, W) + b
yhat.shape

All we have to do is to learn the right weights, and we have a linear model. 
We could easily generate a 2 dimensional output instead of a 1 dimensional output by increasing the dimension of $W$

In [ ]:
W = torch.normal(0., 0.1, (m, 2))
yhat = torch.matmul(X, W)
yhat.shape

## 2 Non linear models
However, the simplicity of linear models is both their strength and weakness:
- The simplicity protects against overfitting
- The simplicity will underfit on non-linear datasets

While a lot of things can be approached as a linear model, there are often more complex and non-linear relations in datasets. A naive idea could be to chain multiple linear models together, like this:

$$
y = f_2(f_1(x))
$$

which is equal to:

$$
y = W_2 (W_1 X + b_1) + b_2
$$

The idea is simply to feed the output of one model to the next. However, the problem here will be that you will end up with something that is equal to a single linear model:

\begin{align}
y &= W_2 W_1 X + W_2 b_1 + b_2\\
y &= W_3 X + b_3\\
\end{align}

If you simplify $W_2 W_1$ to $W_3$ and $W_2 b_1 + b_2$ to $b_3$. 

## 3 Activation functions
However, what will work for chaining is to add an activation function. There are a lot of functions that will work as an activation function; the main requirement is that they are non-linear.

In [ ]:
def sigmoid(x: torch.Tensor) -> torch.Tensor:
    return 1 / (1 + torch.exp(-x))

In [ ]:
k = 100
x = torch.linspace(-5,5, k)
sns.lineplot(x=x, y=sigmoid(x))

Intuitively, you can understand an activation function as a threshold. For example, the sensation of your chair usually lies below the threshold for noticing the sensation. If you increase the input (on the x-axis) you will get above the threshold as output (on the y-axis) at some point.  

There are many types of activation functions. The simplest one is the ReLu, which is 0 if $x \le 0$ and the identity function otherwise.

In [ ]:
def relu(x: torch.Tensor) -> torch.Tensor:
    return torch.maximum(torch.Tensor([0.]), x)

sns.lineplot(x=x, y=relu(x))

This gives us all the building blocks needed for a neural network:

![neural net](https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1547672259/2_i1cdwq.png)

Here, every observation has $n$ features, in this case three.
Every input has a corresponding weight $w$, which is summed up with a linear model. 
The output of the linear model is the fed through an activation function.

Let's test that. First we will generate Tuples $(W, b)$. We will make sure the dimensions of output and input will match.

In [ ]:
import torch
from typing import Tuple

def init_weights(
    m: int, n: int, scale: float = 1e-1, seed: int = 42
) -> Tuple[torch.Tensor, torch.Tensor]:

    torch.manual_seed(seed)

    W = scale * torch.normal(0.0, scale, (m, n))
    b = scale * torch.normal(0.0, scale, (n,))
    return W, b

In [ ]:
W,b = init_weights(10, 2)
W.shape, b.shape

In [ ]:
from typing import List
def torch_network(
    sizes: List[int],
    scale: float = 1e-1,
    seed: int = 42,
) -> List[Tuple[torch.Tensor, torch.Tensor]]:

    params = [init_weights(m, n, scale, seed) for m, n in zip(sizes[:-1], sizes[1:])]

    return params

In [ ]:

layersizes = [2, 8, 2, 1]
params = torch_network(layersizes, seed=42)
for w,_ in params:
    print(w.shape)

In [ ]:
for w,b in params:
    print(w, "\n")
    print(b, "\n")

We have three matrices. The first receives 2 features as input, 8 as output with dimensions $(2,8)$, the second will receive these 8 features as input, 2 as output and thus has $(8,2)$ dimensions, and the final layer transforms 2 dimensions into 1.

This would similar to $f_1(f_2(f_3(x)))$, so chaining 3 models. However, because chaining linear models adds nothing, this could be reduced to $f(x)$ where $f$ is a linear model.

Let's visualize this. First, I will build a 2D grid on the domain $[0,1]$ with a granularity of 100, so I will have $100*100$ coordinates.

In [ ]:
import numpy as np
def build_grid(k: int) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """creates a grid on [0,1] domain
    with a granularity k

    Args:
        k (int): granularity

    Returns:
        np.ndarray: 2D coordinate grid on [0,1]
    """
    x = np.linspace(0, 1, k)
    y = np.linspace(0, 1, k)
    xv, yv = np.meshgrid(x, y)
    grid = np.c_[xv.ravel(), yv.ravel()]
    return grid, x, y

In [ ]:
k = 100
grid, x, y = build_grid(k=k)

In [ ]:
grid[:3] # grid has coordinates

In [ ]:
grid[-3:] # from (0,0) up to (1,1)

In [ ]:
x.shape, x[:5] # x and y contain k values on the range [0,1]

In [ ]:
grid = torch.from_numpy(grid).type(torch.float32)
W, b = params[0]
W.dtype

In [ ]:
from loguru import logger
def predict(params, activations):
    for w, b in params:
        outputs = torch.matmul(activations, w) + b
        logger.info(f"Shape: {outputs.shape}")
        activations = outputs

    return outputs

In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

out = predict(params, grid)
z = out.reshape(k,k)

import plotly.graph_objects as go
fig = go.Figure(data=go.Surface(x=x, y=y, z=z))
fig.show()


Exactly what we expected. We have a 2D hyperplane. If we are doing classification, this hyperplane splits the 3D datapoints into two classes, and we could classify the group of point that fall above or below the hyperplane as belonging to one or another class.

In the case of regression, we would want the point to fall on the hyperplane, as much as possible.

Even though we could scale this to more dimensions easily, the limitations of a linear model should be clear: a lot of data simply is not linear.

To avoid this, we can add tricks to our model:

- a simple approach would be to do basis expansion. E.g. points $X = (x_1, x_2)$ might not be linear serparable, but $\phi(X) = (x_1, x_2, x_1 x_2, x_1^2, x_2^2)$ is a basis expansion that might be linear.
- because basis expansion runs into scaling problems pretty fast (it scales roughly $O(n^2)$) we could use the kernel trick, like a Support Vector Machine does. 
- We could use a decision tree or randomm model, which is a way to chain linear models that result in non linear models) we could use the kernel trick, like a Support Vector Machine does. 
- We could use a decision tree or randomm model, which is a way to chain linear models that result in non linear models.

These "tricks" cover most of the classic models you can find in the scikit-learn library. While they have advantages, there will still be cases where these models are not commplex enough. 

# Neural Networks
A neural network chains linear models with an activation function. The only thing that is different in the next model is that after every linear transformation we will apply the relu function.

![full network](../../reports/figures/nnet.png)

This is what a fully connected neural network would look like.
We can recognize the single units with the multiple inputs.

Rebuild the parameters a few times and see how the model changes.

In [ ]:
layersizes = [2, 8, 16, 4, 1]
random_seed = 5
params = torch_network(layersizes, scale=1e-1, seed=random_seed)

In [ ]:
params[2][0].shape

In [ ]:
def nn_predict(params, activations):
    for w, b in params[:-1]:
        outputs = torch.matmul(activations, w) + b
        logger.info(f"Shape: {outputs.shape}")
        activations = relu(outputs)

    final_w, final_b = params[-1]
    logits = torch.matmul(activations, final_w) + final_b
    logger.info(f"Shape: {logits.shape}")
    return logits

In [ ]:
out = nn_predict(params, grid)
z = out.reshape(k,k)
fig = go.Figure(data=go.Surface(x=x, y=y, z=z))
fig.show()

Et voila! We have a non-linear model. We will need to add a learning process, so that the model can learn to adjust its weights in order to fit the training data, but this is the basics of a neural network.

Change the random_seed to see the impact of different starting values.
Also, experiment with the sizes and shapes of the weights.


- **TAKE AWAY 1**: A neural network is a combination of linear models and activation functions
- **TAKE AWAY 2**: Neural networks are in theory *universal function approximators*. So never NEVER never say that a combination of Linear layers + activation functions is a "simple model". They can approximate EVERY function, you just need to make the network deep & wide enough and train it for long enough!
- **TAKE AWAY 3**: However, you CAN make neural networks very simple. The simplest case is a single linear layer with no activation, which is just a linear model. If you reduce the amount of units, the model will also be simpler.